<a href="https://colab.research.google.com/github/kantopoke/MiniBlockGame/blob/main/%E3%83%9F%E3%83%8B%E3%83%96%E3%83%AD%E3%83%83%E3%82%AF%E5%B4%A9%E3%81%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from IPython.display import HTML
HTML("""
<div style="font-family: system-ui, sans-serif; max-width: 460px">
  <h3 style="margin:6px 0">Breakout (Colab/HTML版)</h3>
  <canvas id="brk" width="420" height="520" style="border:1px solid #999; border-radius:8px; display:block;"></canvas>
  <div style="margin:8px 0; display:flex; gap:8px; flex-wrap:wrap;">
    <button id="bStart">▶ Start / Resume</button>
    <button id="bPause">⏸ Pause</button>
    <button id="bReset">↻ Reset</button>
    <span id="bInfo" style="margin-left:auto; align-self:center;">Click canvas, ← → to move</span>
  </div>
  <div style="display:flex; gap:6px; margin-bottom:6px;">
    <button id="bLeft">←</button>
    <button id="bRight">→</button>
  </div>
</div>
<script>
(() => {
  const cvs = document.getElementById('brk');
  const ctx = cvs.getContext('2d');
  const W = cvs.width, H = cvs.height;

  let paddleW = 100, paddleH = 12, paddleX = (W - paddleW)/2;
  let ballX, ballY, vx, vy, r = 7;
  let bricks, rows=4, cols=7, bw=50, bh=18, pad=10, offX=25, offY=80;
  let running=false, timer=null, lives=2, score=0, gameOver=false, win=false;

  function makeBricks(){
    bricks = [];
    for(let r=0;r<rows;r++){
      for(let c=0;c<cols;c++){
        bricks.push({x:offX + c*(bw+pad), y:offY + r*(bh+pad), alive:true});
      }
    }
  }
  function resetBall(){
    ballX = W/2; ballY = H-90; vx = (Math.random()<0.5?-2:2); vy = -3;
  }
  function reset(){
    paddleX = (W - paddleW)/2;
    score = 0; lives = 2; gameOver=false; win=false;
    makeBricks(); resetBall(); draw();
  }

  function draw(){
    ctx.clearRect(0,0,W,H);
    // bg
    ctx.fillStyle="#fafafa"; ctx.fillRect(0,0,W,H);

    // bricks
    bricks.forEach(b=>{
      if(!b.alive) return;
      ctx.fillStyle="#74b9ff";
      ctx.fillRect(b.x,b.y,bw,bh);
      ctx.strokeStyle="#2980b9"; ctx.strokeRect(b.x,b.y,bw,bh);
    });

    // paddle
    ctx.fillStyle="#2c3e50"; ctx.fillRect(paddleX, H-40, paddleW, paddleH);

    // ball
    ctx.fillStyle="#e74c3c";
    ctx.beginPath(); ctx.arc(ballX, ballY, r, 0, Math.PI*2); ctx.fill();

    // HUD
    ctx.fillStyle="#333"; ctx.font="14px system-ui";
    ctx.fillText("Score: "+score, 10, 20);
    ctx.fillText("Lives: "+lives, W-80, 20);

    if(gameOver || win){
      ctx.fillStyle="rgba(0,0,0,0.5)"; ctx.fillRect(0,0,W,H);
      ctx.fillStyle="#fff"; ctx.textAlign="center";
      ctx.font="bold 22px system-ui";
      ctx.fillText(win? "You Win!":"Game Over", W/2, H/2 - 6);
      ctx.font="16px system-ui";
      ctx.fillText("Score: "+score, W/2, H/2 + 18);
      ctx.textAlign="start";
    }
  }

  function step(){
    if(gameOver || win) return;
    // move ball
    ballX += vx; ballY += vy;

    // walls
    if(ballX<r){ ballX=r; vx*=-1; }
    if(ballX>W-r){ ballX=W-r; vx*=-1; }
    if(ballY<r){ ballY=r; vy*=-1; }

    // paddle
    if(ballY>H-40-r && ballY<H-40+paddleH && ballX>paddleX && ballX<paddleX+paddleW && vy>0){
      vy*=-1;
      // English: hit offset adds side spin
      const center = paddleX + paddleW/2;
      vx += 0.25 * Math.sign(ballX - center);
    }

    // bottom -> life down
    if(ballY>H+r){
      lives--;
      if(lives<0){ stop(); gameOver=true; }
      else { resetBall(); stop(); }
    }

    // bricks
    for(const b of bricks){
      if(!b.alive) continue;
      if(ballX> b.x && ballX< b.x+bw && ballY> b.y && ballY< b.y+bh){
        b.alive=false; vy*=-1; score+=10;
        if(!bricks.some(x=>x.alive)){ win=true; stop(); }
        break;
      }
    }
    draw();
  }

  function start(){ if(!running){ running=true; timer=setInterval(step, 16); info("Running… (← → / buttons)"); } }
  function stop(){ running=false; if(timer){ clearInterval(timer); timer=null; } }
  function info(t){ document.getElementById('bInfo').textContent=t; }

  // controls
  document.getElementById('bStart').onclick = ()=>{ if(!gameOver && !win) start(); };
  document.getElementById('bPause').onclick = ()=>{ stop(); info("Paused."); };
  document.getElementById('bReset').onclick = ()=>{ stop(); reset(); info("Ready. Click canvas then use keys."); };
  document.getElementById('bLeft').onclick = ()=>{ paddleX = Math.max(0, paddleX-20); draw(); };
  document.getElementById('bRight').onclick = ()=>{ paddleX = Math.min(W-paddleW, paddleX+20); draw(); };

  window.addEventListener('keydown', e=>{
    if(e.key==="ArrowLeft"){ paddleX = Math.max(0, paddleX-20); }
    if(e.key==="ArrowRight"){ paddleX = Math.min(W-paddleW, paddleX+20); }
  });

  cvs.addEventListener('click', ()=> cvs.focus(), {passive:true});
  reset(); info("Ready. Click canvas then use ← →");
})();
</script>
""")
